In [2]:
from utils import read_tables
import numpy as np
import pandas
from sdv import Metadata
from sdv.utils import display_tables
from sdv.relational import HMA1

## Import data

In [12]:
real_data = read_tables(
    '../data/rossman_store_sales/source', split_by="_", name_index=0)
del real_data['train']
del real_data['test']

/Users/martinjurkovic/Documents/MAG1-local/Project/Synthetic-data-generation-project/notebooks/utils.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(
/Users/martinjurkovic/Documents/MAG1-local/Project/Synthetic-data-generation-project/notebooks/utils.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(


In [13]:
real_data['store'].columns

Index(['Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [14]:
# make all column names lowercase for all tables in real and synthetic data
for table_name, table in real_data.items():
    real_data[table_name].columns = [col.lower() for col in table.columns]
# for table_name, table in synthetic_data.items():
#     synthetic_data[table_name].columns = [col.lower() for col in table.columns]

# rename date column to sale_date for all tables in real and synthetic data
for table_name, table in real_data.items():
    real_data[table_name] = table.rename(columns={'date': 'sale_date'})
# for table_name, table in synthetic_data.items():
#     synthetic_data[table_name] = table.rename(columns={'date': 'sale_date'})

# add index column of 1 to n to sales table in real and synthetic data
for table_name, table in real_data.items():
    if table_name == 'sales':
        real_data[table_name]['index'] = range(1, len(table) + 1)

# for table_name, table in synthetic_data.items():
#     if table_name == 'sales':
#         synthetic_data[table_name]['index'] = range(1, len(table) + 1)

In [15]:
metadata = Metadata('../models/rossman-sdv-1.0-metadata.json')

In [16]:
display_tables(real_data)   

In [17]:
model = HMA1(metadata=metadata)
model.fit(real_data)

/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/rdt/transformers/numerical.py:100: UserWarning: No rounding scheme detected for column '__sales__store__covariance__0__0'. Data will not be rounded.
  warnings.warn(
/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/rdt/transformers/numerical.py:100: UserWarning: No rounding scheme detected for column '__sales__store__covariance__1__0'. Data will not be rounded.
  warnings.warn(
/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/rdt/transformers/numerical.py:100: UserWarning: No rounding scheme detected for column '__sales__store__covariance__1__1'. Data will not be rounded.
  warnings.warn(
/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/rdt/transformers/numerical.py:100: UserWarning: No rounding scheme detected for column '__sales__store__covariance__2__0'. Data will not be rounde

In [18]:
model_name = '../models/rossman-sdv-1.0.pkl'
model.save(model_name)

In [19]:
synthetic_data = model.sample()

In [20]:
synthetic_data['sales'].to_csv('../data/rossman_store_sales/synthetic_sdv/sales.csv', index=False)
synthetic_data['store'].to_csv('../data/rossman_store_sales/synthetic_sdv/store.csv', index=False)

In [21]:
real_data['sales'].to_csv('../data/rossman_store_sales/source_sdv/sales.csv', index=False)
real_data['store'].to_csv('../data/rossman_store_sales/source_sdv/store.csv', index=False)